# COVID-19 Interactive Analysis Dashboard

In [12]:
# importing libraries

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [13]:
# loading data right from the source:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
countryflag = pd.read_csv('CountryFlag.csv') 

In [14]:
# data cleaning

# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
# ONLY GET THE COUNTRIES FOR DROPDOWN
country_list_df = country_df[['country']]

In [15]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
#active_total = int(country_df['active'].sum())
#DROP ACTIVE BECAUSE I THINK NOT SO ACCURATE
del country_df['active']
country_df = country_df.merge(countryflag, left_on='country', right_on='Country Name', how='left')
del country_df['Country Name']
del country_df['region']


In [16]:
country_df = country_df.rename(columns={'Image URL': 'flag'})

In [17]:
# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

# COVID-19 Confirmed/Death/Recovered cases by countries

## Slide the top countries you want view

In [18]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: yellow'
    g = 'background-color: lightgreen'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="20" >'

def show_latest_cases(n):
    top = int(n)
    #return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)
    pd.set_option('display.max_colwidth', -1)
    #HTML (country_df.to_html(escape=False ,formatters=dict(flag=path_to_image_html)))
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

#interact(show_latest_cases, n='10')
interact(show_latest_cases, n=[10,50,100,150,200])
# SLIDER FOR THE LIST (EASIER TO FILTER)
#interact(show_latest_cases, n=widgets.IntSlider(min=5, max=200, step=1, value=10))

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Dropdown(description='n', options=(10, 50, 100, 150, 200), value=10), Output()), _dom_cl…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [19]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [20]:


def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases for " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 900,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();


# Check the details of any Country or the World

* Select any country from dropdown list for total Confirmed vs Death cases across the timeline

In [21]:
#MAKE COUNTRIES AS DROPDOWN TO EASE FILTER BY COUNTRY + 'WORLD'
country_list_df=country_list_df.sort_values(['country'])
data = []
data.insert(0,{'country':'World'})
country_list_df = pd.concat([pd.DataFrame(data), country_list_df], ignore_index=True)
#interact(plot_cases_of_a_country, country='World')
interact(plot_cases_of_a_country, country=widgets.Dropdown(options=country_list_df['country'], value='World'))

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

# 10 worst hit countries - Confirmed cases

interactive(children=(Dropdown(description='country', options=('World', 'Afghanistan', 'Albania', 'Algeria', '…

    'data': [], 'layout': {'autosize': True, 'template': '...'}
}),), layout=Lay…

# Global spread of COVID-19

In [22]:

world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    if pd.isna(confirmed_df.iloc[i]['state']):
        folium.Circle(
            location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
            fill=True,
            radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
            color='red',
            fill_color='indigo',
            tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                        "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                        "<hr style='margin:10px;color: white;'>"+
                        "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                            "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                            "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                            "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                        "</ul></div>",
            ).add_to(world_map)
    else:
        folium.Circle(
            location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
            fill=True,
            radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
            color='red',
            fill_color='indigo',
            tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                        "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                        "<h5 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['state'] + "</h5>"            
                        "<hr style='margin:10px;color: white;'>"+
                        "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                            "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                            "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                            "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                        "</ul></div>",
            ).add_to(world_map)        
world_map
